# Ministral 3 Models: Compact Multimodal AI on Amazon Bedrock

---

[Ministral 3](https://mistral.ai/news/mistral-3) models are part of Mistral's latest model family, offering compact yet powerful options for various deployment scenarios. These models share the multimodal and multilingual capabilities of the larger Mistral 3 family while maintaining smaller footprints for edge and cost-efficient deployments.

## Available Models

| Model | Model ID | Parameters | Best For |
|-------|----------|------------|----------|
| **Ministral 3B** | `mistral.ministral-3-3b-instruct` | 3B | Edge devices, ultra-low latency |
| **Ministral 8B** | `mistral.ministral-3-8b-instruct` | 8B | Balanced performance/efficiency |
| **Ministral 14B** | `mistral.ministral-3-14b-instruct` | 14B | Higher accuracy, complex reasoning |

## Key Highlights

- **Multimodal**: Image understanding capabilities
- **Multilingual**: Support for 40+ languages
- **Apache 2.0**: Fully open-source license
- **Best Performance-to-Cost Ratio**: Optimized for their parameter class
- **Edge-Ready**: Designed for local and on-device deployment

In this notebook, we demonstrate Ministral capabilities using Amazon Bedrock:

1. Setup and model comparison
2. Basic text generation
3. Reasoning and problem solving
4. Code generation
5. Structured output (JSON)
6. Vision and image understanding

---

## Getting Started

---

In [ ]:
# Install required dependencies
%pip install --upgrade --quiet boto3 botocore

In [ ]:
import boto3
import json
import time
from botocore.exceptions import ClientError

In [ ]:
# Configuration
REGION = "us-west-2"

# Ministral Model IDs
MODELS = {
    "3B": "mistral.ministral-3-3b-instruct",
    "8B": "mistral.ministral-3-8b-instruct",
    "14B": "mistral.ministral-3-14b-instruct"
}

# Initialize Bedrock client
bedrock_client = boto3.client(
    service_name='bedrock-runtime',
    region_name=REGION
)

print("Available Ministral Models:")
for name, model_id in MODELS.items():
    print(f"  {name}: {model_id}")

In [ ]:
def converse(model_id, messages, system_prompt=None, max_tokens=1024, temperature=0.7):
    """
    Helper function to interact with Ministral models using the Converse API.
    
    Args:
        model_id: The Ministral model ID to use
        messages: List of message dictionaries with 'role' and 'content'
        system_prompt: Optional system prompt string
        max_tokens: Maximum tokens to generate
        temperature: Sampling temperature (0.0 to 1.0)
    
    Returns:
        Response from the model
    """
    converse_params = {
        "modelId": model_id,
        "messages": messages,
        "inferenceConfig": {
            "maxTokens": max_tokens,
            "temperature": temperature
        }
    }
    
    if system_prompt:
        converse_params["system"] = [{"text": system_prompt}]
    
    response = bedrock_client.converse(**converse_params)
    return response


def get_response_text(response):
    """Extract text content from a Converse API response."""
    return response['output']['message']['content'][0]['text']

---

## 1. Model Comparison

Let's compare response quality and latency across all three Ministral models.

---

In [ ]:
# Compare all three models on the same prompt
test_prompt = "Explain what an API is in 2-3 sentences, suitable for a beginner."

print("Comparing Ministral Models")
print("="*70)
print(f"Prompt: {test_prompt}")
print("="*70)

for name, model_id in MODELS.items():
    print(f"\n--- Ministral {name} ---")
    
    messages = [{"role": "user", "content": [{"text": test_prompt}]}]
    
    start_time = time.time()
    response = converse(model_id, messages, temperature=0.3)
    latency = time.time() - start_time
    
    print(f"Latency: {latency:.2f}s")
    print(f"Response: {get_response_text(response)}")

---

## 2. Basic Text Generation

Demonstrating text generation capabilities with the Ministral 8B model (balanced choice).

---

In [ ]:
# Use 8B as the default for examples
DEFAULT_MODEL = MODELS["8B"]

# Question answering
messages = [
    {
        "role": "user",
        "content": [{"text": "What are the three laws of thermodynamics? Explain each briefly."}]
    }
]

response = converse(DEFAULT_MODEL, messages, temperature=0.3)
print(get_response_text(response))

In [ ]:
# Creative writing
creative_prompt = "Write a haiku about cloud computing."

messages = [{"role": "user", "content": [{"text": creative_prompt}]}]

response = converse(DEFAULT_MODEL, messages, temperature=0.8)
print(get_response_text(response))

---

## 3. Reasoning and Problem Solving

Testing logical reasoning capabilities across model sizes.

---

In [ ]:
# Math word problem
math_problem = """
A store sells apples for $2 each and oranges for $3 each. 
If Sarah buys 5 apples and 3 oranges, and pays with a $20 bill, 
how much change does she receive?

Show your work step by step.
"""

messages = [{"role": "user", "content": [{"text": math_problem}]}]

print("Math Problem - Ministral 8B")
print("="*50)
response = converse(MODELS["8B"], messages, temperature=0.1)
print(get_response_text(response))

In [ ]:
# Logic puzzle - compare 8B vs 14B
logic_puzzle = """
If all roses are flowers, and some flowers fade quickly, 
can we conclude that some roses fade quickly?

Explain your reasoning.
"""

messages = [{"role": "user", "content": [{"text": logic_puzzle}]}]

print("Logic Puzzle Comparison")
print("="*70)

for name in ["8B", "14B"]:
    print(f"\n--- Ministral {name} ---")
    response = converse(MODELS[name], messages, temperature=0.1)
    print(get_response_text(response))

---

## 4. Code Generation

Ministral models can generate code for common programming tasks.

---

In [ ]:
# Simple code generation
code_prompt = """
Write a Python function that checks if a string is a palindrome.
Include a docstring and handle edge cases.
"""

messages = [{"role": "user", "content": [{"text": code_prompt}]}]

response = converse(MODELS["8B"], messages, temperature=0.1)
print(get_response_text(response))

In [ ]:
# Code explanation
code_to_explain = """
Explain what this Python code does:

```python
def mystery(n):
    return n if n <= 1 else mystery(n-1) + mystery(n-2)
```
"""

messages = [{"role": "user", "content": [{"text": code_to_explain}]}]

response = converse(MODELS["8B"], messages, temperature=0.1)
print(get_response_text(response))

---

## 5. Structured Output (JSON)

Ministral models can generate structured JSON output, useful for data extraction and API responses.

---

In [ ]:
# JSON extraction
json_prompt = """
Extract the following information from this text and return it as JSON:

"John Smith is a 32-year-old software engineer from Seattle. 
He has 8 years of experience and specializes in Python and AWS."

Return a JSON object with keys: name, age, occupation, location, experience_years, skills
"""

messages = [{"role": "user", "content": [{"text": json_prompt}]}]

system_prompt = "You are a helpful assistant that returns only valid JSON without any additional text."

response = converse(MODELS["8B"], messages, system_prompt=system_prompt, temperature=0.1)
print(get_response_text(response))

In [ ]:
# Sentiment analysis with structured output
sentiment_prompt = """
Analyze the sentiment of these customer reviews and return JSON:

1. "The product arrived quickly and works perfectly. Love it!"
2. "Terrible experience. The item was broken and customer service was unhelpful."
3. "It's okay, nothing special but does the job."

Return a JSON array with objects containing: review_number, sentiment (positive/negative/neutral), confidence (high/medium/low)
"""

messages = [{"role": "user", "content": [{"text": sentiment_prompt}]}]

response = converse(MODELS["8B"], messages, system_prompt=system_prompt, temperature=0.1)
print(get_response_text(response))

---

## 6. Vision and Image Understanding

Ministral models support multimodal capabilities including image understanding.

> **Note**: For vision tasks, use the **InvokeModel API** with Mistral's native message format. Converse API support for images is coming soon.

---

In [ ]:
import base64
import urllib.request

def load_image_as_base64(image_path_or_url):
    """Load an image from a file path or URL and convert to base64."""
    if image_path_or_url.startswith(('http://', 'https://')):
        req = urllib.request.Request(
            image_path_or_url,
            headers={'User-Agent': 'Mozilla/5.0'}
        )
        with urllib.request.urlopen(req) as response:
            image_data = response.read()
    else:
        with open(image_path_or_url, 'rb') as f:
            image_data = f.read()
    
    return base64.standard_b64encode(image_data).decode('utf-8')


def analyze_image(model_id, image_source, prompt, media_type="image/jpeg"):
    """
    Analyze an image using Ministral's vision capabilities via InvokeModel API.
    
    Args:
        model_id: The Ministral model ID to use
        image_source: File path or URL to the image
        prompt: Text prompt describing what to analyze
        media_type: MIME type of the image
    
    Returns:
        Model's analysis of the image
    """
    image_base64 = load_image_as_base64(image_source)
    
    # Mistral's native multimodal format
    payload = {
        "messages": [
            {
                "role": "user",
                "content": [
                    {
                        "type": "image_url",
                        "image_url": {
                            "url": f"data:{media_type};base64,{image_base64}"
                        }
                    },
                    {
                        "type": "text",
                        "text": prompt
                    }
                ]
            }
        ],
        "max_tokens": 1024,
        "temperature": 0.3
    }
    
    response = bedrock_client.invoke_model(
        modelId=model_id,
        body=json.dumps(payload)
    )
    
    result = json.loads(response['body'].read())
    return result['choices'][0]['message']['content']

In [ ]:
# Image analysis with Ministral 8B
image_path = "../Pixtral-samples/Pixtral_data/Amazon_Chart.png"

analysis_prompt = """Analyze this chart and provide:
1. What type of data is being shown
2. Key metrics and their values
3. Notable trends or patterns
"""

print("Image Analysis - Ministral 8B")
print("="*50)
result = analyze_image(MODELS["8B"], image_path, analysis_prompt, media_type="image/png")
print(result)

In [ ]:
# Compare vision capabilities across model sizes
image_path = "../Pixtral-samples/Pixtral_data/Crosstab_of_Cola_Preference_by_Age_and_Gender.png"

table_prompt = """Extract the key insights from this table:
1. What data categories are shown?
2. What are the total values by gender?
3. Which age group has the highest frequency?
"""

print("Vision Comparison - Table Analysis")
print("="*70)

for name in ["8B", "14B"]:
    print(f"\n--- Ministral {name} ---")
    start_time = time.time()
    result = analyze_image(MODELS[name], image_path, table_prompt, media_type="image/png")
    latency = time.time() - start_time
    print(f"Latency: {latency:.2f}s")
    print(f"Response: {result[:500]}...")  # Truncate for readability

---

## Choosing the Right Model

| Use Case | Recommended Model | Why |
|----------|-------------------|-----|
| Simple Q&A, classification | **Ministral 3B** | Fastest, lowest cost |
| General purpose, balanced tasks | **Ministral 8B** | Good performance/cost ratio |
| Complex reasoning, code generation | **Ministral 14B** | Higher capability |

---

## Conclusion

Ministral models offer a compelling option for cost-sensitive and latency-critical applications:

- **Ministral 3B**: Ultra-efficient for simple tasks and edge deployments
- **Ministral 8B**: Balanced performance for most general use cases
- **Ministral 14B**: Enhanced capabilities for more demanding tasks

All three models support the Converse API on Amazon Bedrock, making them easy to integrate into existing applications.

---

## Clean Up

This notebook uses Amazon Bedrock's serverless inference, so there are no resources to clean up. You are charged based on usage (input and output tokens processed).